In [2]:
!pip install torch torchvision torchaudio


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
torch.cuda.is_available()

False

In [3]:
import transformers
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW, BitsAndBytesConfig
from tqdm.notebook import tqdm

# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [2]:
!pip install transformers
!pip install torch
!pip install tpdm

     ---------------------------------------- 8.8/8.8 MB 11.0 MB/s eta 0:00:00
     ------------------------------------- 388.6/388.6 kB 11.8 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 11.7 MB/s eta 0:00:00
     -------------------------------------- 269.5/269.5 kB 8.1 MB/s eta 0:00:00
     -------------------------------------- 269.5/269.5 kB 8.4 MB/s eta 0:00:00
     ------------------------------------- 172.0/172.0 kB 10.1 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 198.6/198.6 MB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 4.9 MB/s eta 0:00:00
     ------------------------------------- 536.2/536.2 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.13.0 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.10.1 which is incompatible.
tensorflow-intel 2.13.0 requires tensorflow-estimator<2.14,>=2.13.0, but you have tensorflow-estimator 2.10.0 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requireme

In [4]:
!pip install torch torchvision torchaudio
!pip install datasets

     ---------------------------------------- 1.2/1.2 MB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 7.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 510.5/510.5 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 24.8/24.8 MB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 170.9/170.9 kB ? eta 0:00:00
     -------------------------------------- 365.2/365.2 kB 4.6 MB/s eta 0:00:00
     -------------------------------------- 116.3/116.3 kB 3.4 MB/s eta 0:00:00
     -------------------------------------- 134.8/134.8 kB 4.0 MB/s eta 0:00:00
     -------------------------------------- 50.4/50.4 kB 198.0 kB/s eta 0:00:00
     ---------------------------------------- 76.4/76.4 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.1
    Uninstalling fsspec-2024.3.1:
      Successfully uninstalled fsspec-2024.3.1



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1. 데이터 수집 및 준비

In [6]:
train = pd.read_csv('./df.csv').iloc[:,1:]
val = pd.read_csv('./df_val.csv').iloc[:,1:]
train.head()

,speaker_id,form,standard_form,dialect_form,isDialect
0,2.0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도,False
1,1.0,어어 그니까,어어 그니까,어어 그니까,False
2,2.0,아 잘도,아 잘도,아 잘도,False
3,1.0,예초기 뱀도 짤려,예초기 뱀도 짤려,예초기 뱀도 짤려,False
4,2.0,어 그니까 완전 끔찍해,어 그니까 완전 끔찍해,어 그니까 완전 끔찍해,False


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159636 entries, 0 to 159635
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   speaker_id     159633 non-null  float64
 1   form           159636 non-null  object 
 2   standard_form  157798 non-null  object 
 3   dialect_form   157800 non-null  object 
 4   isDialect      159636 non-null  bool   
dtypes: bool(1), float64(1), object(3)
memory usage: 5.0+ MB


In [8]:
train = train.loc[train['standard_form'].notnull()]

In [9]:
# speaker_id는 모델 학습에 사용하지 않기 때문에 결측치가 있어도 넘어갑니다.
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 157798 entries, 0 to 159635
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   speaker_id     157795 non-null  float64
 1   form           157798 non-null  object 
 2   standard_form  157798 non-null  object 
 3   dialect_form   157798 non-null  object 
 4   isDialect      157798 non-null  bool   
dtypes: bool(1), float64(1), object(3)
memory usage: 6.2+ MB


# 2. 모델 선택 및 수정
- 모델은 KoBART 모델을 사용하고 싶음
- 한국어로 학습되어 있고 무게가 그렇게 무겁지 않아 사용하기에 적절하다고 판단
- URL : gogamza/kobart-base-v2

In [10]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v2")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-base-v2")

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

C:\Users\gudtj\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gudtj\.cache\huggingface\hub\models--gogamza--kobart-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

# 3. 모델 학습 및 fine-tuning

In [12]:
! pip install -U accelerate
! pip install -U transformers

import accelerate
import transformers

transformers.__version__, accelerate.__version__

     -------------------------------------- 290.1/290.1 kB 9.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


('4.39.1', '0.28.0')

In [13]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='../chatbot/',          # output directory for model checkpoints
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=1,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# Implement a subclass of `Dataset` to use with `Trainer`
from torch.utils.data import Dataset

class TranslationDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=128):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        source_text = self.data.iloc[idx]['dialect_form']
        target_text = self.data.iloc[idx]['standard_form']
        source = self.tokenizer(source_text, return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True)
        target = self.tokenizer(target_text, return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True)
        source_ids = source["input_ids"].squeeze()
        target_ids = target["input_ids"].squeeze()
        return {"input_ids": source_ids, "labels": target_ids}

# Instantiate the dataset
train_dataset = TranslationDataset(tokenizer, train)
val_dataset = TranslationDataset(tokenizer, val)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()
